في الاول هنعمل كريت ل الداتا ونخزنها في مونجو دي بي 

In [ ]:
from pymongo import MongoClient
import random

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["my_database"]

# ----- Prepare first and last names -----
first_names = ["Ahmed", "Ali", "Sara", "Mona", "Omar", "Youssef", "Nour", "Laila", "Khaled", "Hana"]
last_names = ["Refat", "Hassan", "Ibrahim", "Mohamed", "Farouk", "Adel", "Sami", "Nabil", "Tarek", "Fathy"]

# ----- Generate Users -----
users = []
for i in range(1, 21):  # 20 users
    user = {
        "user_id": i,
        "name": f"{random.choice(first_names)} {random.choice(last_names)}",
        "email": f"user{i}@example.com",
        "age": random.randint(18, 60)
    }
    users.append(user)

# Insert users
db["users"].insert_many(users)
print("Inserted 20 users")

# ----- Generate Products -----
products = []
categories = ["Electronics", "Books", "Clothes", "Sports"]
for i in range(1, 21):  # 20 products
    product = {
        "product_id": i,
        "name": f"Product {i}",
        "category": random.choice(categories),
        "price": round(random.uniform(10, 500), 2)
    }
    products.append(product)

# Insert products
db["products"].insert_many(products)
print("Inserted 20 products")







reviews = []
for i in range(1, 21):  # 20 reviews
    review = {
        "review_id": i,
        "user_id": random.randint(1, 20),
        "product_id": random.randint(1, 20),
        "rating": random.randint(1, 5),
        "comment": f"This is a comment for product {random.randint(1, 20)}"
    }
    reviews.append(review)

# Insert reviews
db["reviews"].insert_many(reviews)
print("Inserted 20 reviews")


orders = []
for i in range(1, 21):  # 20 orders
    order = {
        "order_id": i,
        "user_id": random.randint(1, 20),  # user_id موجود
        "products": [
            {"product_id": random.randint(1, 20), "quantity": random.randint(1, 5)}
            for _ in range(random.randint(1, 3))  # كل order فيها 1-3 منتجات
        ],
        "total": 0  # هنحسبها بعدين
    }
    # حساب total
    order["total"] = sum([p["quantity"] * round(random.uniform(10, 500), 2) for p in order["products"]])
    orders.append(order)

# Insert orders
db["orders"].insert_many(orders)
print("Inserted 20 orders")
orders = []




Inserted 20 users
Inserted 20 products


In [ ]:
#  create csv files 
import pandas as pd
import random

# table 1
users_sql = pd.DataFrame({
    "user_id": range(1, 21),
    "name": [f"User_{i}" for i in range(1, 21)],
    "email": [f"user{i}@sql.com" for i in range(1, 21)],
    "signup_date": pd.date_range("2025-01-01", periods=20)
})
users_sql.to_csv("users_sql.csv", index=False)


# table 2
products_sql = pd.DataFrame({
    "product_id": range(1, 21),
    "name": [f"Product_{i}" for i in range(1, 21)],
    "category": [random.choice(["Electronics", "Books", "Clothing"]) for _ in range(20)],
    "price": [round(random.uniform(10, 500), 2) for _ in range(20)]
})
products_sql.to_csv("products_sql.csv", index=False)

# table 3

orders_sql = []
for i in range(1, 21):
    orders_sql.append({
        "order_id": i,
        "user_id": random.randint(1, 20),
        "product_id": random.randint(1, 20),
        "quantity": random.randint(1, 5)
    })

orders_sql_df = pd.DataFrame(orders_sql)
orders_sql_df.to_csv("orders_sql.csv", index=False)


# table 4

reviews_sql = []
for i in range(1, 21):
    reviews_sql.append({
        "review_id": i,
        "user_id": random.randint(1, 20),
        "product_id": random.randint(1, 20),
        "rating": random.randint(1, 5),
        "comment": f"This is SQL comment for product {random.randint(1, 20)}"
    })

reviews_sql_df = pd.DataFrame(reviews_sql)
reviews_sql_df.to_csv("reviews_sql.csv", index=False)


intrgation of data and the processing 
---------------------

In [25]:
# اول خطوه قراءه البيانات من mongodb

from pymongo import MongoClient
import pandas as pd
from pyspark.sql   import SparkSession

# create  sparksession 

spark = (
    SparkSession.builder
    .appName("firt_ETL")
    .config("spark.jars", "/home/ahmed-refat/Desktop/end_to_end_projects/postgresql-42.2.5.jar")
    .getOrCreate()
)



# the connection of mongodb local 
client = MongoClient("mongodb://localhost:27017/")
db = client["my_database"] # دي اسم الداتا بيز بتاعتي في مونجو دي بي 

collection1= pd.DataFrame(list(db["users"].find())).drop(columns=["_id"])
collection2= pd.DataFrame(list(db["orders"].find())).drop(columns=["_id"])
collection3= pd.DataFrame(list(db["products"].find())).drop(columns=["_id"])




# collection قراءه ال 
df_users =spark.createDataFrame(collection1)
df_orders =spark.createDataFrame(collection2)
df_products =spark.createDataFrame(collection3)


df_users.show(5)
df_orders.show(5)
df_products.show(5)







+-------+-------------+-----------------+---+
|user_id|         name|            email|age|
+-------+-------------+-----------------+---+
|      1|  Ali Mohamed|user1@example.com| 38|
|      2|    Hana Sami|user2@example.com| 51|
|      3|   Ahmed Sami|user3@example.com| 28|
|      4|Khaled Hassan|user4@example.com| 18|
|      5|  Khaled Adel|user5@example.com| 29|
+-------+-------------+-----------------+---+
only showing top 5 rows


+--------+-------+--------------------+------------------+
|order_id|user_id|            products|             total|
+--------+-------+--------------------+------------------+
|       1|     14|[{quantity -> 4, ...|           4749.09|
|       2|     15|[{quantity -> 2, ...|            757.82|
|       3|     14|[{quantity -> 1, ...|           1215.49|
|       4|      7|[{quantity -> 3, ...|            827.63|
|       5|     12|[{quantity -> 5, ...|2769.6800000000003|
+--------+-------+--------------------+------------------+
only showing top 5 rows
+----------+---------+--------+------+
|product_id|     name|category| price|
+----------+---------+--------+------+
|         1|Product 1|   Books|438.83|
|         2|Product 2|   Books| 44.64|
|         3|Product 3| Clothes| 86.02|
|         4|Product 4|   Books|327.74|
|         5|Product 5|  Sports|335.79|
+----------+---------+--------+------+
only showing top 5 rows


In [16]:
# هنا بقا هنعمل عمليه تحويل الملفات csv  ل داتا فريم جوا باي سبارك 

# قراءة ملفات CSV في PySpark
df_users_csv = spark.read.csv("users_sql.csv", header=True, inferSchema=True)
df_products_csv = spark.read.csv("products_sql.csv", header=True, inferSchema=True)
df_orders_csv = spark.read.csv("orders_sql.csv", header=True, inferSchema=True)
df_reviews_csv = spark.read.csv("reviews_sql.csv", header=True, inferSchema=True)

# عرض أول 5 صفوف لأي ملف للتأكد

df_orders_csv.show(5)
df_reviews_csv.show(5)


+--------+-------+----------+--------+
|order_id|user_id|product_id|quantity|
+--------+-------+----------+--------+
|       1|      9|         8|       4|
|       2|      3|         7|       2|
|       3|      6|        16|       2|
|       4|      6|        14|       1|
|       5|      3|        18|       1|
+--------+-------+----------+--------+
only showing top 5 rows
+---------+-------+----------+------+--------------------+
|review_id|user_id|product_id|rating|             comment|
+---------+-------+----------+------+--------------------+
|        1|     11|        13|     2|This is SQL comme...|
|        2|      4|        11|     2|This is SQL comme...|
|        3|     11|         1|     2|This is SQL comme...|
|        4|      2|        13|     1|This is SQL comme...|
|        5|      8|         6|     4|This is SQL comme...|
+---------+-------+----------+------+--------------------+
only showing top 5 rows


In [17]:
# عمليه دمج الداتا فريمز المتشابها في داتا فريم واحده عن طريق حذف المكرر وعمل جوين عليهم 
csv_df_clean = df_users_csv.drop("name", "email")

df_users_clean = df_users.join(
    csv_df_clean,
    on="user_id",
    how="left"
)

# دمج جداول المنتجات 
# لما جيت اجرب لقيت الاتنين نفس الحاجه ف هنشتغل علي الجدول الي جي من مونجو دي بي وخلاص 
df_product_clean= df_products


# دمج جدولين ال orders 
# بس هيظهر قدمنا مشكله ان الجدول الي جي من المونجو فيه مشكله 
# وتحديدا ان في عمود اسمه برودكت جواه list 



df_orders.printSchema()
df_orders_csv.printSchema()



root
 |-- order_id: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: long (valueContainsNull = true)
 |-- total: double (nullable = true)

root
 |-- order_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)



In [18]:
# create explode to the column of product in df_orders == mongodb
from pyspark.sql.functions import col , explode 
df_explode= (df_orders.withColumn("product",explode("products")))

df_orders_explode=(
  df_explode
  .withColumn("prouct_id",col("product")["product_id"])
  .withColumn("quantity",col("product")["quantity"])
  .drop("product", "products")

)
#df_orders_explode.show()


In [24]:
# connect with postgres 
df_users_clean.write.jdbc(
    url="jdbc:postgresql://localhost:5432/ecommerce_dw",
    table="dim_users",
    mode="append",
    properties={
        "user": "admin",
        "password": "admin",
        "driver": "org.postgresql.Driver"
    }
)




Py4JJavaError: An error occurred while calling o446.jdbc.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:592)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:47)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:112)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:112)
	at scala.Option.foreach(Option.scala:437)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:112)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:272)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:276)
	at or

In [23]:
df_users_clean.write.jdbc(
    url=url,
    table="dim_users",
    mode="append",
    properties=properties
)


Py4JJavaError: An error occurred while calling o442.jdbc.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:592)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:47)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:112)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:112)
	at scala.Option.foreach(Option.scala:437)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:112)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:272)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:276)
	at or